# AzureML in a Day

データサイエンティストがAzure Machine Learning (Azure ML)を使用してモデルをトレーニングし、その後予測にモデルを使用する方法を学びます。このチュートリアルでは、Azure MLの基本概念とその一般的な使用方法に慣れることができます。

*トレーニングスクリプト*を実行するために*コマンドジョブ*を送信し、スクリプトを実行するために必要な*ジョブ環境*を設定する方法を学びます。

トレーニングスクリプトはデータの準備を行い、その後モデルをトレーニングして登録します。モデルを取得したら、それを*エンドポイント*として*デプロイ*し、*推論*のためにエンドポイントを呼び出します。

実行する手順は次の通りです：

> * Azure MLワークスペースに接続する
> * ジョブ環境を作成する
> * トレーニングスクリプトを作成する
> * 適切なジョブ環境で設定されたトレーニングスクリプトを実行するコマンドジョブを作成して実行する
> * トレーニングスクリプトの出力を確認する
> * 新しくトレーニングしたモデルをエンドポイントとしてデプロイする
> * 推論のためにAzure MLエンドポイントを呼び出す

## 前提条件

* Azureサブスクリプション。Azureサブスクリプションをお持ちでない場合は、[無料アカウントを作成](https://aka.ms/AMLFree)してから始めてください。
* Azure MLワークスペースの動作環境。ワークスペースはAzureポータル、Azure CLI、またはPython SDKを使用して作成できます。[詳しくはこちら](https://docs.microsoft.com/azure/machine-learning/how-to-manage-workspace?tabs=python)を参照してください。
* Azure Machine Learning [ワークスペース]()
* ワークスペースと計算インスタンス。これらは[クイックスタート: Azure Machine Learningの使い方を始める](https://docs.microsoft.com/azure/machine-learning/quickstart-create-resources#create-compute-instance)を完了することで作成できます。

## ワークスペースに接続する

コードに取り組む前に、Azure MLワークスペースに接続する必要があります。ワークスペースはAzure Machine Learningのトップレベルリソースであり、Azure Machine Learningを使用して作成するすべてのアーティファクトを一元的に管理する場所を提供します。

ワークスペースへのアクセスには `DefaultAzureCredential` を使用します。
`DefaultAzureCredential` は、ほとんどのAzure SDK認証シナリオを処理するために使用されます。

もしうまくいかない場合は、他の利用可能な資格情報についてのリファレンスを参照してください: [資格情報の設定例](../../configuration.ipynb)、[azure-identityリファレンスドキュメント](https://docs.microsoft.com/python/api/azure-identity/azure.identity?view=azure-python)。

In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

If you want to use a browser to login and authenticate, you can use the following code instead. In this example, you'll use the `DefaultAzureCredential`.

In [ ]:
# Handle to the workspace
# from azure.ai.ml import MLClient

# Authentication package
# from azure.identity import InteractiveBrowserCredential
# credential = InteractiveBrowserCredential()

次のセルに、サブスクリプションID、リソースグループ名、およびワークスペース名を入力します。これらの値を見つけるには：

Azure Machine Learning Studioの右上ツールバーで、ワークスペース名を選択します。
ワークスペース、リソースグループ、サブスクリプションIDの値をコードにコピーします。
一つの値をコピーしてエリアを閉じ、貼り付けた後、次の値を取得するために戻ってください。

![image of workspace credentials](media\find-credentials.png)

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<AML_WORKSPACE_NAME>",
)

結果として、他のリソースやジョブを管理するために使用するワークスペースへのハンドラーが作成されます。

> [!IMPORTANT]
> MLClientを作成することは、ワークスペースへの接続を意味しません。クライアントの初期化は遅延評価され、最初の呼び出しが行われるまで待機します（以下のノートブックでは、ジョブ環境の作成時にこの呼び出しが行われます）。

## ジョブ環境を作成する

AzureMLジョブを実行するには、[環境](https://docs.microsoft.com/azure/machine-learning/concept-environments)が必要です。環境は、トレーニングを行うコンピュート上にインストールしたいソフトウェアランタイムとライブラリをリストアップします。これはローカルマシン上のPython環境に似ています。

AzureMLは、多くのキュレーション済みまたは既製の環境を提供しており、一般的なトレーニングや推論のシナリオに役立ちます。また、DockerイメージやConda構成を使用して独自のカスタム環境を作成することもできます。

この例では、Conda yamlファイルを使用してジョブのカスタムConda環境を作成します。

まず、ファイルを保存するディレクトリを作成します。

In [3]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

次に、dependenciesディレクトリにファイルを作成します。下のセルは、IPythonマジックでファイルを作成したディレクトリに書き込んでいます。

In [25]:
%%writefile {dependencies_dir}/conda.yaml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - mlflow-skinny==2.3.2
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

Overwriting ./dependencies/conda.yaml


このyamlファイルに指定した定義に従って、ワークスペースにカスタム環境を作成および登録します：

In [26]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults pipeline",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name aml-scikit-learn is registered to workspace, the environment version is 5


## コマンドジョブとは？

Azure MLの*コマンドジョブ*を作成して、クレジットデフォルト予測のモデルをトレーニングします。コマンドジョブは、指定された環境でサーバーレスコンピュート上で*トレーニングスクリプト*を実行するために使用されます。環境はすでに作成済みですので、次にトレーニングスクリプトを作成します。

*トレーニングスクリプト*は、データの準備、トレーニング、およびトレーニング済みモデルの登録を行います。このチュートリアルでは、Pythonのトレーニングスクリプトを作成します。

コマンドジョブはCLI、Python SDK、またはスタジオインターフェースから実行できます。このチュートリアルでは、Azure ML Python SDK v2を使用してコマンドジョブを作成および実行します。

トレーニングジョブの実行後、モデルをデプロイし、予測を行います。

## トレーニングスクリプトを作成する

まず、トレーニングスクリプトである*main.py*というPythonファイルを作成します。

最初に、スクリプトのためのソースフォルダを作成します。

In [6]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

このスクリプトは、データの前処理を行い、それをテストデータとトレーニングデータに分割します。その後、このデータを使用してツリーベースのモデルをトレーニングし、出力モデルを返します。

パイプライン実行中にパラメーターとメトリクスをログするために、[MLFlow](https://learn.microsoft.com/azure/machine-learning/how-to-log-mlflow-models)を使用します。

以下のセルでは、IPythonのマジックコマンドを使用して、トレーニングスクリプトを先ほど作成したディレクトリに書き込みます。

In [29]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
   
    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    ###################
    #<prepare the data>
    ###################
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    
    credit_df = pd.read_csv(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    train_df, test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )
    ####################
    #</prepare the data>
    ####################

    ##################
    #<train the model>
    ##################
    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    ###################
    #</train the model>
    ###################

    ##########################
    #<save and register model>
    ##########################
    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    ###########################
    #</save and register model>
    ###########################
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/main.py


このスクリプトでは、モデルがトレーニングされると、モデルファイルが保存され、ワークスペースに登録されます。これで、登録されたモデルを推論エンドポイントで使用できるようになります。

## コマンドの設定

必要なタスクを実行できるスクリプトが作成できたので、コマンドラインアクションを実行できる汎用的な**コマンド**を使用します。このコマンドラインアクションは、システムコマンドを直接呼び出すか、スクリプトを実行することで実現できます。

ここでは、入力データ、分割比率、学習率、および登録されたモデル名を指定するための入力変数を作成します。コマンドスクリプトは次のことを行います：
* 以前に作成した環境を使用します。コマンドを実行する際に最新バージョンの環境を示すために `@latest` 表記を使用できます。
* 表示名、実験名などのメタデータを設定します。*実験*は、特定のプロジェクトに対して行うすべてのイテレーションのコンテナです。同じ実験名で送信されたすべてのジョブは、Azure ML Studioで隣り合わせにリストされます。
* コマンドラインアクション自体を設定します - この場合は `python main.py` です。入力/出力は `${{ ... }}` 表記でコマンド内でアクセスできます。
* このサンプルでは、インターネット上のファイルからデータにアクセスします。

In [30]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "credit_defaults_model"

job = command(
    inputs=dict(
        data=Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default%20of%20credit%20card%20clients.csv",
        ),
        test_train_ratio=0.2,
        learning_rate=0.25,
        registered_model_name=registered_model_name,
    ),
    code="./src/",  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment="aml-scikit-learn@latest", # Created at above cell
    experiment_name="train_model_credit_default_prediction",
    display_name="credit_default_prediction",
)

## ジョブを送信する

AzureMLで実行するためにジョブを送信する時が来ました。今回は、`ml_client.jobs` の `create_or_update` を使用します。

In [31]:
ml_client.create_or_update(job)

Uploading src (0.0 MBs): 100%|██████████| 2905/2905 [00:00<00:00, 123366.88it/s]




Experiment,Name,Type,Status,Details Page
train_model_credit_default_prediction,happy_heart_c5pmt7fn7l,command,Starting,Link to Azure Machine Learning studio


## ジョブの出力を確認し、ジョブの完了を待つ

前のセルの出力にあるリンクを選択して、Azure ML Studioでジョブを確認します。

このジョブの出力はAzure ML Studioで次のように表示されます。メトリクス、出力などの詳細を確認するには、各タブを探索してください。完了すると、ジョブはトレーニングの結果としてワークスペースにモデルを登録します。 

![Screenshot that shows the job overview](media/view-job.gif "View the job in studio")

> [!IMPORTANT]
> Wait until the status of the job is complete before returning to this notebook to continue. The job will take 2 to 3 minutes to run. It could take longer (up to 10 minutes) if the compute has been scaled down to zero nodes and custom environment is still building.



## モデルをオンラインエンドポイントとしてデプロイする

機械学習モデルをAzureクラウドのWebサービス、つまり[`オンラインエンドポイント`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints)としてデプロイします。

機械学習サービスをデプロイするには通常、次のものが必要です：

* デプロイしたいモデル資産（ファイル、メタデータ）。これらの資産はトレーニングジョブで既に登録済みです。
* サービスとして実行するコード。コードは、指定された入力リクエストに対してモデルを実行します。このエントリースクリプトは、デプロイされたWebサービスに送信されたデータを受け取り、それをモデルに渡してからモデルの応答をクライアントに返します。スクリプトはモデルに特有のものです。エントリースクリプトは、モデルが期待するデータを理解し、モデルが返すデータを理解する必要があります。MLFlowモデルの場合、このチュートリアルのように、このスクリプトは自動的に作成されます。スコアリングスクリプトのサンプルは[こちら](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online)にあります。

## 新しいオンラインエンドポイントを作成する

登録済みのモデルと推論スクリプトが揃ったので、オンラインエンドポイントを作成する時が来ました。エンドポイント名はAzureリージョン全体で一意である必要があります。このチュートリアルでは、`UUID`を使用して一意の名前を作成します。

In [32]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

> [!NOTE]
> Expect the endpoint creation to take approximately 6 to 8 minutes.

In [33]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint credit-endpoint-7c595fb9 provisioning state: Succeeded


エンドポイントを作成したら、以下のようにして情報を取得できます。

In [34]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "credit-endpoint-7c595fb9" with provisioning state "Succeeded" is retrieved


## モデルをエンドポイントにデプロイする

エンドポイントが作成されたら、エントリースクリプトを使用してモデルをデプロイします。各エンドポイントには複数のデプロイメントを持つことができます。これらのデプロイメントへのトラフィックの振り分けは、ルールを使用して指定できます。ここでは、すべての受信トラフィックを処理する単一のデプロイメントを作成します。デプロイメントには、任意のカラー名（例：*blue*、*green*、*red*）を選択します。

登録されたモデルの最新バージョンを確認するには、Azure ML Studioの**モデル**ページを確認することができます。あるいは、以下のコードで最新バージョン番号を取得することもできます。

In [35]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

モデルの最新バージョンをデプロイします。

> [!NOTE]
> Expect this deployment to take approximately 6 to 8 minutes.

In [36]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint credit-endpoint-7c595fb9 exists


............................................................................................................

### サンプルクエリでテストする

モデルがエンドポイントにデプロイされたので、推論を実行できます。

スコアスクリプトのrunメソッドで期待される形式に従って、サンプルリクエストファイルを作成します。

In [37]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [38]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

Writing ./deploy/sample-request.json


In [39]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

'[1, 0]'

## リソースのクリーンアップ

エンドポイントを使用しない場合は、リソースの利用を停止するためにエンドポイントを削除します。削除する前に、他のデプロイメントがエンドポイントを使用していないことを確認してください。

> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [40]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

...................................................................................................

## Next Steps

Learn about creating a multi step pipeline for this script [Create production ML pipelines in a Jupyter notebook](https://github.com/Azure/azureml-examples/blob/main/tutorials/e2e-ds-experience/e2e-ml-workflow.ipynb).